In [9]:
# Does what it says on the tin.
# Note that it takes a while to read all the mdb thingies.
# To run pandas_access successfully, install mdbtools through whatever you need.
# -Matt

import pandas
import numpy as np
import pandas_access

DATASET = "./FDOC_January_2017.mdb"



Inmate_release_root : Basic inmate information on inmates released since 10-1-1997

Inmate_release_aliases: Aliases for these released inmates

Inmate_release_offenses_CPS: Current prison offenses for these released inmates

Inmate_release_offenses_prpr: Prior prison offenses for these released inmates

Inmate_release_residence: Release plan address for these released inmates

Inmate_release_detainers: Detainers for these released inmates

Inmate_release_incarhist: Incarceration history for these released inmates

Inmate_release_scarsmarks: Tattoos for these released inmates

Inmate_active_root:  Basic inmate information on active inmates

Inmate_active_aliases: Aliases for these active inmates

Inmate_active_offenses_CPS: Current prison offenses for these active inmates

Inmate_active_offenses_prpr: Prior prison offenses for these active inmates

Inmate_active_detainers: Detainers for these active inmates

Inmate_active_incarhist: Incarceration history for these active inmates

Inmate_active_scarsmarks: Tattoos for these active inmates

Offender_root: Basic offender information on current community supervision offenders

Offender_aliases: Aliases for offenders

Offender_offenses_CCS: Current community supervision offenses for offenders

Offender_residence: Most recent address (excluding confidential addresses) for offenders


All sealed and expunged records have been deleted from the datasets. No confidential data is included.

In [10]:
table_list = []
for tablename in pandas_access.list_tables(DATASET):
    print(tablename)
    table_list.append(tablename)

tables_dict = {}

for table in table_list: #doing more makes it take forever.
    tables_dict[table] = pandas_access.read_table(DATASET, table, low_memory = False)



CONTENTS
INMATE_RELEASE_ROOT
INMATE_RELEASE_ALIASES
INMATE_RELEASE_OFFENSES_CPS
INMATE_RELEASE_OFFENSES_prpr
INMATE_RELEASE_RESIDENCE
INMATE_RELEASE_DETAINERS
INMATE_RELEASE_INCARHIST
INMATE_RELEASE_SCARSMARKS
OFFENDER_ROOT
OFFENDER_ALIASES
OFFENDER_OFFENSES_CCS
OFFENDER_RESIDENCE
INMATE_ACTIVE_ROOT
INMATE_ACTIVE_ALIASES
INMATE_ACTIVE_OFFENSES_CPS
INMATE_ACTIVE_OFFENSES_prpr
INMATE_ACTIVE_DETAINERS
INMATE_ACTIVE_INCARHIST
INMATE_ACTIVE_SCARSMARKS


In [11]:
tables_dict.keys()

[u'INMATE_RELEASE_INCARHIST',
 u'INMATE_ACTIVE_ROOT',
 u'INMATE_RELEASE_DETAINERS',
 u'OFFENDER_OFFENSES_CCS',
 u'INMATE_RELEASE_OFFENSES_CPS',
 u'INMATE_RELEASE_RESIDENCE',
 u'INMATE_RELEASE_SCARSMARKS',
 u'INMATE_ACTIVE_ALIASES',
 u'INMATE_ACTIVE_DETAINERS',
 u'INMATE_RELEASE_OFFENSES_prpr',
 u'INMATE_ACTIVE_SCARSMARKS',
 u'OFFENDER_ROOT',
 u'OFFENDER_RESIDENCE',
 u'INMATE_ACTIVE_OFFENSES_prpr',
 u'OFFENDER_ALIASES',
 u'INMATE_RELEASE_ROOT',
 u'INMATE_ACTIVE_INCARHIST',
 u'INMATE_ACTIVE_OFFENSES_CPS',
 u'CONTENTS',
 u'INMATE_RELEASE_ALIASES']

In [12]:
reoffend_with_scars = []
no_reoffend_with_scars = []
reoffend_without_scars = []
no_reoffend_without_scars = []

In [24]:
# tables_dict["INMATE_ACTIVE_SCARSMARKS"] # check if multiple different lastnames and such?...

In [14]:
id_set = set()
names_hist = {}


for index, row in tables_dict["OFFENDER_ALIASES"].iterrows():
    if row.DCNumber not in id_set:
        id_set.add(row.DCNumber)
        names_hist[row.FirstName] = names_hist.get(row.FirstName, default=0) + 1

In [42]:
dcnumber_to_scarsmarks = {}
for index, row in tables_dict["INMATE_ACTIVE_SCARSMARKS"].iterrows():
    this_tup = (row.Location, row.Description)
    if row.Type == "TATTOO":
        if row.DCNumber in dcnumber_to_scarsmarks:
            dcnumber_to_scarsmarks[row.DCNumber].append(this_tup)
        else:
            dcnumber_to_scarsmarks[row.DCNumber] = [this_tup]


In [45]:
# sorted(names_hist.items(), key=lambda x: x[1], reverse=True) #perhaps compare to overall name frequency?
# sm = tables_dict["INMATE_ACTIVE_SCARSMARKS"]
# sm[sm.Location == "HEAD"]
# dcnumber_to_scarsmarks H12939

In [46]:
dcnumber_to_offenses = {}
for index, row in tables_dict["INMATE_ACTIVE_OFFENSES_CPS"].iterrows(): #current sentence
    if row.DCNumber in dcnumber_to_offenses:
        dcnumber_to_offenses[row.DCNumber].append(row.adjudicationcharge_descr)
    else:
        dcnumber_to_offenses[row.DCNumber] = [row.adjudicationcharge_descr]

In [49]:
dcnumber_to_offenses[dcnumber_to_scarsmarks.keys()[1]]

['SELL/ETC/COC/ETC 1K FT.EDU',
 'POSS.CONTROL.SUBS/OTHER',
 'FEL/DELI W/GUN/CONC WPN/AMMO',
 'POSSESS MDMA',
 'POSS.MARIJUANA OVR 20 GRAMS']

In [64]:
def contains_string(target_string, some_list, substring=True):
    if substring:
        for entry in some_list:
#             print entry, '\t', target_string, '\t', (target_string in entry)
            if (target_string in entry):
                return True
    else:
        for entry in some_list:
            if entry == target_string:
                return True
    return False

def contains_value(target_value, some_list):
    for entry in some_list:
        if (target_value == entry) or (target_value in entry): #not sure if this is the best format.
            return True
    return False

    

In [67]:
murder_head = []
murder_nohead = []
nomurder_head = []
nomurder_nohead = []

for dcn in dcnumber_to_offenses.keys():
    sm = dcnumber_to_scarsmarks.get(dcn, [])
    offs = dcnumber_to_offenses[dcn]
    has_headneck = ( contains_value("HEAD", sm) or contains_value("FACE", sm) or contains_value("NECK", sm) )
    has_murder = contains_string("MUR", offs) # need to make sure all mur = murder and vice versa.
    if has_headneck:
        if has_murder:
            murder_head.append(dcn)
        else:
            nomurder_head.append(dcn)
    else:
        if has_murder:
            murder_nohead.append(dcn)
        else:
            nomurder_nohead.append(dcn)

In [98]:
number_head = len(murder_head) + len(nomurder_head)
number_nohead = len(murder_nohead) + len(nomurder_nohead)

prob_murder_head = len(murder_head) / float(number_head)
prob_murder_nohead = len(murder_nohead) / float(number_nohead)

In [99]:
prob_murder_head

0.11210864253650775

In [100]:
prob_murder_nohead

0.15315853658536585

In [102]:
t_percent = (100 * ( (prob_murder_nohead / prob_murder_head) - 1))
do_or_dont = (prob_murder_nohead > prob_murder_head)
if do_or_dont:
    t_percent = (100 * ( (prob_murder_nohead / prob_murder_head) - 1))
    t_comparison = "*don't*"
else:
    t_percent = (100 * ( (prob_murder_head / prob_murder_nohead) - 1))
    t_comparison = "*do*"


tstring = ("The probability that an inmate has committed or attempted murder "
            "is %2.1f%% greater if they %s have a head/face/neck tattoo" % (t_percent,t_comparison))
print(tstring)

The probability that an inmate has committed or attempted murder is 36.6% greater if they *don't* have a head/face/neck tattoo
